This notebook was used by Jeremy to create summaries of all the lesson transcripts.

In [ ]:
import requests,sys,time,os,openai
from pathlib import Path

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def prompt(s,p, mx=512, max_retries=3):
    m=[{"role": "system", "content": s}, {"role": "user", "content": p}]
    for attempt in range(1, max_retries + 1):
        try:
            res = openai.ChatCompletion.create(messages=m, model="gpt-4", temperature=0.2, top_p=0.2, max_tokens=mx)
            print(res['usage'])
            return res['choices'][0]['message']['content']
        except openai.error.RateLimitError as e:
            if attempt < max_retries:
                retry_after = int(e.headers.get("Retry-After", 1))  # Use the Retry-After header or default to 1 second
                print(f"429 error: Too Many Requests. Retrying in {retry_after} seconds (Attempt {attempt}/{max_retries})")
                time.sleep(retry_after)
            else: raise

In [ ]:
# prompt("Just testing", "Is this thing on?", 100)

In [ ]:
def txt_rng(x):
    s = x.split(' ')
    c = len(s)
    j = len(' '.join(s[:4000]))
    if j>len(x)-100: return len(x)
    t = x[:j]
    return len(t) - t[-1:0:-1].find(' .') - 1

In [ ]:
fn = Path('lesson09.txt')
txt = fn.read_text()

In [ ]:
len(txt),txt_rng(txt)

(79698, 20450)

In [ ]:
def summs(fn):
    fn = Path(fn)
    x = fn.read_text()
    i,res = 0,''
    while len(x)>50:
        j = txt_rng(x)
        if i==0: pre='The text contains the transcript of the first section of a video lesson. There are other later parts not included in the text.'
        elif j>=len(x)-50: pre='The text contains the last section of a video lesson. There are other earlier parts not included in the text.'
        else: pre='The text contains the middle section of a video lesson. There are other earlier and later parts not included in the text.'
        pre += ' Summarise the section of the lesson in the text in 3-4 paragraphs.\n\ntext:\n\n'
        res += prompt(s_s, pre + x[:j])
        print(i,len(x[:j]), len(x), len(x[:j].split()))
#         print(pre)
        x = x[j+1:]
        i=j
#         time.sleep(1)
    with open(fn.stem + f"-summ0.txt", "w") as f: f.write(res)
    return res

In [ ]:
files = ('lesson11.txt','lesson12.txt','lesson13.txt','lesson14.txt','lesson15.txt','lesson16.txt','lesson17.txt','lesson18.txt',
         'lesson19.txt','lesson20.txt','lesson21.txt','lesson22.txt','lesson23.txt','lesson24.txt')

In [ ]:
for f in files: summs(f)

In [ ]:
j_s = """You are an AI language model trained to summarise the fast.ai lesson taught by Jeremy Howard 
described in the input. First, you provide a 2-4 word descriptive title as a markdown heading.
Then you provide a 3-4 paragraph summary of the text written in in an informal style like Jeremy Howard -- here is a sample of his style:
"Today we finish off our study of collaborative filtering by looking closely at embeddings—a critical building block of many
deep learning algorithms. Then we’ll dive into convolutional neural networks (CNNs) and see how they really work. We’ve used
plenty of CNNs through this course, but we haven’t peeked inside them to see what’s really going on in there. 
As well as learning about their most fundamental building block, the convolution, we’ll also look at pooling, dropout, and more."
Finally you provide a list of all the key skills and concepts taught in the lesson, formatted as a markdown list."""

In [ ]:
files = ('lesson13-summ0.txt','lesson14-summ0.txt','lesson15-summ0.txt','lesson16-summ0.txt','lesson17-summ0.txt','lesson18-summ0.txt',
         'lesson19-summ0.txt','lesson20-summ0.txt','lesson21-summ0.txt','lesson22-summ0.txt','lesson23-summ0.txt','lesson24-summ0.txt')

for f in files:
    print('--',f,'--')
    print(prompt(j_s, Path(f).read_text()))

In [ ]:
s_s = "You are a system for creating summaries of input text. You do not produce any output other than the summary requested by the user."